In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')
df.dtypes
df = df.rename(columns={'Personalized_Recommendation_Frequency': 'Personalized_Recommendation_Purchase'})


In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Gender'] = df['Gender'].astype('category')
df['Personalized_Recommendation_Purchase'] = df['Personalized_Recommendation_Purchase'].astype('category')
df['Review_Left'] = df['Review_Left'].astype('category')
df['Review_Helpfulness'] = df['Review_Helpfulness'].astype('category')
df['Review_Reliability'] = df['Review_Reliability'].astype('category')
df['Recommendation_Helpfulness'] = df['Recommendation_Helpfulness'].astype('category')
df['Add_to_Cart_Browsing'] = df['Add_to_Cart_Browsing'].astype('category')
df['Cart_Completion_Frequency'] = df['Cart_Completion_Frequency'].astype('category')
df['Purchase_Categories'] = df['Purchase_Categories'].astype('category')
df['Purchase_Frequency'] = df['Purchase_Frequency'].astype('category')
df['Browsing_Frequency'] = df['Browsing_Frequency'].astype('category')
df['Product_Search_Method'] = df['Product_Search_Method'].astype('category')
df['Search_Result_Exploration'] = df['Search_Result_Exploration'].astype('category')
df['Saveforlater_Frequency'] = df['Saveforlater_Frequency'].astype('category')

In [ ]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_report = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})
display(missing_report[missing_report['Missing Values'] > 0])

In [ ]:
custom_missing = ["Nothing", ".", "Nil"]
df.replace(custom_missing, pd.NA, inplace=True)

In [ ]:
columns_to_check = df.select_dtypes(include=['float64', 'int64'])

def find_outliers_pandas(data, column):
	Q1 = data[column].quantile(0.25)
	Q3 = data[column].quantile(0.75)
	IQR = Q3 - Q1
	lower_bound = Q1 - 1.5 * IQR
	upper_bound = Q3 + 1.5 * IQR
	outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
	return outliers

outliers_dict = {}

for column in columns_to_check:
	outliers_dict[column] = find_outliers_pandas(df, column)

for column, outliers in outliers_dict.items():
	print(f"Outliers in '{column}':")
	print(outliers)
	print("\n")